# Modelo B 


por: Catalina Alvarez 

# Seccion 1: Setup del cuaderno

In [4]:
import pandas as pd
from ydata_profiling import ProfileReport
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib

# import word tokenize
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
    HashingVectorizer,
)

nltk.download("punkt_tab")
import spacy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Load the large spanish model
nlp = spacy.load("es_core_news_lg")
# nlp = spacy.load('es_dep_news_trf')
%matplotlib inline

ModuleNotFoundError: No module named 'ydata_profiling'

# Sección 2 - Entendimiento de los datos 

In [ ]:
df_original = data = pd.read_csv('./datos/fake_news_spanish.csv', sep=';', encoding='utf-8', on_bad_lines='warn')
#df_test_original = pd.read_excel("./datos/fake_news_test.csv")


In [ ]:
df_original

# Calidad de los datos

Dimensiones de calidad de los datos: 
    Consistencia 
    Validez 
    Campletitud 
    Unicidad 

# Completitud

Bajo una inspeccion sencilla, parece no haber problemas de completitud en los datos

In [ ]:
df_original["Label"].value_counts()

In [ ]:
# strip all values in the column
df_original['Descripcion'] = df_original['Descripcion'].str.strip()
# count all values in the column where the value is an empty string

df_original['Descripcion'].isnull().sum()

# Validez 
Se identifican los siguientes problemas de validez: 
Los textos en el dataset tienen problemas, vocales con tilde han sido remplazados por caracteres no validos. 


In [ ]:
def fix_encoding(text):
    replacements = {
        'Ã¡': 'á',
        'Ã©': 'é',
        'Ã­': 'í',
        'Ã³': 'ó',
        'Ãº': 'ú',
        'Ã±': 'ñ'
    }
    
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    
    return text

df_original['Descripcion'] = df_original['Descripcion'].apply(fix_encoding)


In [ ]:
df_original.head(5)

# Consistencia 
Hay entradas entre comillas. Posiblemente esto no genere problemas en el dataset. 

# Unicidad
El dataset no tiene columanas que deben ser unicas. Bajo una sencilla inspeccion, hay filas duplicadas. 



In [ ]:
df_original.duplicated().sum()

In [ ]:
df_original.drop_duplicates(inplace=True)